In [ ]:
import requests
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance

def search_restaurants(api_key, location, term, limit=5, min_rating=4.0):
    headers = {
        'Authorization': f'Bearer {api_key}',
    }
    params = {
        'location': location,
        'term': term,
        'limit': limit,
        'sort_by': 'rating',  
    }
    url = 'https://api.yelp.com/v3/businesses/search'
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        businesses = data['businesses']
        
        if not businesses:
            print(f"No {term} restaurants found in {location}")
            return
        
        print(f"Top {limit} {term} restaurants in {location} with over {min_rating} star ratings:")
        for idx, business in enumerate(businesses[:limit], start=1):
            print(f"{idx}. {business['name']} - Rating: {business['rating']} ({business['review_count']} reviews)")
            print(f"   Address: {', '.join(business['location']['display_address'])}")
            print(f"   Phone: {business['phone']}")
            print(f"   Website: {business['url']}")
            
            details_url = f"https://api.yelp.com/v3/businesses/{business['id']}"
            details_response = requests.get(details_url, headers=headers)
            
            if details_response.status_code == 200:
                details_data = details_response.json()
                
                if 'hours' in details_data:
                    hours = details_data['hours'][0]['open']
                    closing_times = [f"{day['day']}: {day['end']}" for day in hours]
                    print(f"   Closing Times: {' | '.join(closing_times)}")
                else:
                    print("   Opening hours not available.")
                
                if 'coordinates' in business['location']:
                    restaurant_coords = (business['location']['coordinates']['latitude'], business['location']['coordinates']['longitude'])
                    user_coords = get_coordinates(location)
                    if user_coords:
                        dist = haversine(user_coords[0], user_coords[1], restaurant_coords[0], restaurant_coords[1])
                        print(f"   Distance from {location}: {dist:.2f} km")
                    else:
                        print(f"   Distance from {location}: Distance information not available.")
                else:
                    print("   Distance information not available.")
            
            else:
                print(f"   Failed to fetch details for {business['name']}.")
            
            print()

    else:
        print(f"Error fetching data from Yelp API. Status code: {response.status_code}")

def get_coordinates(location):
    return None

def main():
    api_key = '_YnQmHeIT6KafvXf1y2PvZw7kZ1u-QxhcrcyNpJiGJDLOh0L0h7bcyv1MDw8_OYjgeMiBAosZ8DlzSDwqrAVAJLuJl_yCPrw3OEsDjpz2Hr6Wir9zm74QLd8GbfiZXYx'
    
    location = input("Enter your ZIP code: ").strip()
    
    term = input("What type of food are you craving? (e.g., Italian, Mexican, Japanese): ").strip()
    
    search_restaurants(api_key, location, term)

if __name__ == "__main__":
    main()

